In [12]:
import numpy as np
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import re
import xlrd
import copy
from time import *
import  math
import os
import random

In [19]:
#读取输入输出数据
#读取输入数据集
large_input_all_np=np.zeros((5373000,8))
for i in range (1,15):
    large_input_all_np=np.loadtxt("E:\\CPFEM\\CPFEM_ANN\\input_all_"+str(i)+".txt",delimiter=" ")
large_output_all_np=np.load(base_path+"output_y_all.npy")

In [ ]:
#对数据进行分割
input_1=[]  #268
input_2=[]  #408
input_3=[]  #278-398
output_1=[] #268
output_2=[] #408
output_3=[] #278-398
input_1_np=np.zeros((199*200*7,23))
input_2_np=np.zeros((199*200*7,23))
input_3_np=np.zeros((199*200*7*13,23))
output_1_np=np.zeros((199*200*7,1))
output_2_np=np.zeros((199*200*7,1))
output_3_np=np.zeros((199*200*7*13,1))
k=0
j=0
for i in range(0,15*7,15):
    print(k)
    #input_1.append(large_input_all_np[1*15*200*199:-1*15*200*199][i*199*200:(i+1)*199*200].flatten())
    input_1_np[k*199*200:(k+1)*199*200]=large_input_all_np[1*15*200*199:-1*15*200*199][i*199*200:(i+1)*199*200]
    #input_2.append(large_input_all_np[1*15*200*199:-1*15*200*199][(i+14)*199*200:(i+15)*199*200].flatten())
    input_2_np[k*199*200:(k+1)*199*200]=large_input_all_np[1*15*200*199:-1*15*200*199][(i+14)*199*200:(i+15)*199*200]
    #input_3.append(large_input_all_np[1*15*200*199:-1*15*200*199][(i+1)*199*200:(i+14)*199*200].flatten())
    input_3_np[j*199*200:(j+13)*199*200]=large_input_all_np[1*15*200*199:-1*15*200*199][(i+1)*199*200:(i+14)*199*200]
    #output_1.append(large_output_all_np[1*15*200*199:-1*15*200*199][i*199*200:(i+1)*199*200].flatten())
    output_1_np[k*199*200:(k+1)*199*200]=large_output_all_np[1*15*200*199:-1*15*200*199][i*199*200:(i+1)*199*200]
    #output_2.append(large_output_all_np[1*15*200*199:-1*15*200*199][(i+14)*199*200:(i+15)*199*200].flatten())
    output_2_np[k*199*200:(k+1)*199*200]=large_output_all_np[1*15*200*199:-1*15*200*199][(i+14)*199*200:(i+15)*199*200]
    #output_3.append(large_output_all_np[1*15*200*199:-1*15*200*199][(i+1)*199*200:(i+14)*199*200].flatten())
    output_3_np[j*199*200:(j+13)*199*200]=large_output_all_np[1*15*200*199:-1*15*200*199][(i+1)*199*200:(i+14)*199*200]
    k=k+1
    j=j+13

In [ ]:
#训练数据集的建立
large_input_all_np=np.zeros((input_3_np.shape[0],8))
large_input_all_np[:,0]=input_3_np[:,5]
large_input_all_np[:,1]=input_3_np[:,11]/input_3_np[:,18]
large_input_all_np[:,2]=input_3_np[:,17]
large_input_all_np[:,3]=input_3_np[:,19]
large_input_all_np[:,4]=input_3_np[:,20]
large_input_all_np[:,5]=input_3_np[:,21]
large_input_all_np[:,6]=input_3_np[:,22]
large_input_all_np[:,7]=input_3_np[:,11]
#训练数据集归一化的建立
large_input_new=copy.deepcopy(large_input_all_np)
large_output_new=copy.deepcopy(output_3_np)
large_input_new_nor=np.zeros_like(large_input_new)
large_output_new_nor=np.zeros_like(large_output_new)
nor_num=[]
#归一化参数的获取
for i in range(8):
    if i==0:
        print(i,np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)))
        nor_num.append(np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)))
    elif i==1:
        print(i,np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)))
        nor_num.append(np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)))
    elif i>=2: 
        print(i,np.mean(large_input_new[:,i]),np.std(large_input_new[:,i]))
        nor_num.append([np.mean(large_input_new[:,i]),np.std(large_input_new[:,i])])
#对数据进行归一化
for i in range(8):
    if i==0:
        large_input_new_nor[:,i]=np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)/np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5))
        #large_input_new_nor[:,i]=(large_input_new_nor[:,i]-np.mean(large_input_new_nor[:,i]))/np.std(large_input_new_nor[:,i])
    elif i==1:
        large_input_new_nor[:,i]=np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)/np.min(np.log10(large_input_new[:,i]+abs(np.min(large_input_new[:,i]))+1e-5))
        #large_input_new_nor[:,i]=(large_input_new_nor[:,i]-np.mean(large_input_new_nor[:,i]))/np.std(large_input_new_nor[:,i])
    elif i>=2:
        large_input_new_nor[:,i]=(large_input_new[:,i]-np.mean(large_input_new[:,i]))/np.std(large_input_new[:,i])
large_output_new_nor[:,0]=copy.deepcopy(large_output_new[:,0])

In [31]:
N12=199#N_EXTEND
N1=int(len(large_input_new_nor)/N12*0.8)
total_num=range(int(len(large_input_new_nor)/N12))
train_num=np.load(base_path+"train_num.npy").tolist()
test_num=np.load(base_path+"test_num.npy").tolist()

In [63]:
#建立不同的神经网络
hidden_size_4=[[8,8],[16,16],[24,24],[32,32]]
hidden_size_5=[[8,8,8],[16,16,16],[24,24,24],[32,32,32]]
hidden_size_6=[[8,8,8,8],[16,16,16,16],[24,24,24,24],[32,32,32,32]]
hidden_size_7=[[8,8,8,8,8],[16,16,16,16,16],[24,24,24,24,24],[32,32,32,32,32]]

In [ ]:
#不同结构的对比

In [ ]:
##tanh
for i in range(0,1):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            """
            我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            """
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Tanh(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Tanh(),
                nn.Linear(hidden_size[1], output_size),
            )
        def forward(self, x):
            """
            在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            """
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_4[0], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"4_layer_tanh"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_4_layer_tanh"+str(i), loss_append)
print("________________________________________________")
print("The tanh layer has been successfully completed")

In [ ]:
##Relu
for i in range(0,1):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            """
            我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            """
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.ReLU(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.ReLU(),
                nn.Linear(hidden_size[1], output_size),
            )
        def forward(self, x):
            """
            在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            """
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_4[0], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"4_layer_relu"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_4_layer_relu"+str(i), loss_append)
print("________________________________________________")
print("The relu layer has been successfully completed")

In [ ]:
##Sigmoid
for i in range(0,1):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            """
            我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            """
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Sigmoid(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Sigmoid(),
                nn.Linear(hidden_size[1], output_size),
            )
        def forward(self, x):
            """
            在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            """
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_4[0], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"4_layer_sigmoid"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_4_layer_sigmoid"+str(i), loss_append)
print("________________________________________________")
print("The sigmoid layer has been successfully completed")

In [ ]:
#不同网络结构的对比

In [ ]:
#四层网络
for i in range(0,4):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            """
            我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            """
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Tanh(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Tanh(),
                nn.Linear(hidden_size[1], output_size),
            )
        def forward(self, x):
            """
            在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            """
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_4[i], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"4_layer_"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_4_layer"+str(i), loss_append)
print("________________________________________________")
print("The four layer has been successfully completed")

In [ ]:
##五层网络
for i in range(4):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            #我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Tanh(),
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Tanh(),
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.Tanh(),
                nn.Linear(hidden_size[2], output_size),
            )
        def forward(self, x):
            #在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_5[i], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"5_layer_"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_5_layer"+str(i), loss_append)
print("________________________________________________")
print("The five layer has been successfully completed")

In [ ]:
##六层网络
for i in range(3,4):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            #我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Tanh(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Tanh(),
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.Tanh(),
                nn.Linear(hidden_size[2], hidden_size[3]),
                nn.Tanh(),
                nn.Linear(hidden_size[3], output_size),
            )
        def forward(self, x):
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_6[i], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"6_layer_"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_6_layer"+str(i), loss_append)
print("________________________________________________")
print("The six layer has been successfully completed")

In [ ]:
##七层网络
for i in range(3,4):
    loss_append=[]
    class TwoLayerNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            #我们在构建模型的时候，能够使用nn.Sequential的地方，尽量使用它，因为这样可以让结构更加清晰
            super(TwoLayerNet, self).__init__()
            self.twolayernet = nn.Sequential(
                nn.Linear(input_size, hidden_size[0]),
                nn.Tanh(),#nn.Relu()#nn.Tanh()nn.Sigmoid()
                nn.Linear(hidden_size[0], hidden_size[1]),
                nn.Tanh(),
                nn.Linear(hidden_size[1], hidden_size[2]),
                nn.Tanh(),
                nn.Linear(hidden_size[2], hidden_size[3]),
                nn.Tanh(),
                nn.Linear(hidden_size[3], hidden_size[4]),
                nn.Tanh(),
                nn.Linear(hidden_size[4], output_size),
            )
        def forward(self, x):
            """
            在forward函数中，我们会接受一个Variable，然后我们也会返回一个Varible
            """
            y_pred = self.twolayernet(x)
            return y_pred
    epoch_0=32
    input_size=8
    output_size=1
    model = TwoLayerNet(input_size, hidden_size_7[i], output_size)
    loss_fn = nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    EPOCH =int(N1/epoch_0*6000)
    begin_time = time()
    N1=int(len(large_input_new_nor)/N12*0.8)
    for t in range(EPOCH):      
        # 向前传播
        x1=random.sample(train_num,epoch_0)
        z1=np.zeros((N12*epoch_0,input_size))
        z2=np.zeros((N12*epoch_0,1))
        k=0
        for j in x1:
            z1[k*N12:(k+1)*N12]=large_input_new_nor[j*N12:(j+1)*N12]
            z2[k*N12:(k+1)*N12]=large_output_new_nor[j*N12:(j+1)*N12]
            k=k+1
        xx=Variable(torch.from_numpy(z1).to(torch.float32)) 
        yy=Variable(torch.from_numpy(z2).to(torch.float32))  
        y_pred= model(xx)
        # 计算损失
        loss = loss_fn(y_pred, yy)
        # 显示损失
        loss_append.append(loss.item())
        if (t+1) % int(N1/epoch_0*10)== 0:
            print((t+1)/int(N1/epoch_0*10),loss.item())
        # 在我们进行梯度更新之前，先使用optimier对象提供的清除已经积累的梯度。
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 更新梯度
        optimizer.step()
    end_time = time()
    run_time = end_time-begin_time
    print ('该循环程序运行时间:',run_time) #该循环程序运行时间： 1.4201874
    PATH="E:\\CPFEM\\CPFEM_ANN\\Article\\model\\"+"7_layer_"+str(i)+".pt"
    torch.save(model.state_dict(), PATH)
    #loss_total.append(loss_append)
    np.save("E:\\CPFEM\\CPFEM_ANN\\Article\\model\\loss_7_layer"+str(i), loss_append)
print("________________________________________________")
print("The seven layer has been successfully completed")

In [341]:
#测试数据集的误差分析
error_sum=[]
for test_n in train_num:#[9]
    #test_n=N1+200
    text_x=(Variable(torch.from_numpy(large_input_new_nor[test_n*N12:(test_n+1)*N12]).to(torch.float32)))
    test_y=(large_output_new_nor[test_n*N12:(test_n+1)*N12])#*del_stress_std+del_stress_mean
    test_error=np.hstack(((model(text_x).detach().numpy())-test_y))#*del_stress_std+del_stress_mean
    error_sum.append(np.max(abs(test_error)))
error_sum=[]
for test_n in test_num:#[9]
    #test_n=N1+200
    text_x=(Variable(torch.from_numpy(large_input_new_nor[test_n*N12:(test_n+1)*N12]).to(torch.float32)))
    test_y=(large_output_new_nor[test_n*N12:(test_n+1)*N12])#*del_stress_std+del_stress_mean
    test_error=np.hstack(((model(text_x).detach().numpy())-test_y))#*del_stress_std+del_stress_mean
    error_sum.append(np.max(abs(test_error)))
plt.plot(error_sum)

(199, 1)

In [ ]:
#DNN做预测

In [ ]:
#测试数据集
input_1_S_np=copy.deepcopy(large_input_all_np[0:1*15*200*199])  #1E-2
output_1_S_np=copy.deepcopy(large_output_all_np[0:1*15*200*199])
#测试数据集的重新分布
large_input_all_np_1_S=np.zeros((input_1_S_np.shape[0],8))
large_input_all_np_1_S[:,0]=input_1_S_np[:,5]
large_input_all_np_1_S[:,1]=input_1_S_np[:,11]/input_1_S_np[:,18]
large_input_all_np_1_S[:,2]=input_1_S_np[:,17]
large_input_all_np_1_S[:,3]=input_1_S_np[:,19]
large_input_all_np_1_S[:,4]=input_1_S_np[:,20]
large_input_all_np_1_S[:,5]=input_1_S_np[:,21]
large_input_all_np_1_S[:,6]=input_1_S_np[:,22]
large_input_all_np_1_S[:,7]=input_1_S_np[:,11]
#数据归一化
large_input_new_1_S=copy.deepcopy(large_input_all_np_1_S)
large_output_new_1_S=copy.deepcopy(output_1_S_np)
large_input_new_nor_1_S=np.zeros_like(large_input_new_1_S)
large_output_new_nor_1_S=np.zeros_like(large_output_new_1_S)
large_output_new_nor_1_S[:,0]=copy.deepcopy(large_output_new_1_S[:,0])
for i in range(8):
    if i==0:
        large_input_new_nor_1_S[:,i]=np.log10(large_input_new_1_S[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)/nor_num[0]
        #large_input_new_nor[:,i]=(large_input_new_nor[:,i]-np.mean(large_input_new_nor[:,i]))/np.std(large_input_new_nor[:,i])
    elif i==1:
        large_input_new_nor_1_S[:,i]=np.log10(large_input_new_1_S[:,i]+abs(np.min(large_input_new[:,i]))+1e-5)/nor_num[1]
        #large_input_new_nor[:,i]=(large_input_new_nor[:,i]-np.mean(large_input_new_nor[:,i]))/np.std(large_input_new_nor[:,i])
    elif i>=2:
        large_input_new_nor_1_S[:,i]=(large_input_new_1_S[:,i]-nor_num[i][0])/nor_num[i][1]

In [ ]:
#单晶应力的计算
def cal_stress(delt_sigma):
    large_output_new_cal=np.zeros(200)
    for i in range(1,200):
        large_output_new_cal[i]=np.sum(delt_sigma[0:i])
    return large_output_new_cal
#多晶应力的计算
def poly_stress(delt_sigma):
    poly_sum=np.zeros(200)
    for kk in range(200):
        poly_sum=poly_sum+cal_stress(delt_sigma[kk*199:(kk+1)*199])
    return poly_sum/200
#多晶应力的计算
def poly_strain(delt_sigma):
    poly_sum=np.zeros(199)
    for kk in range(200):
        poly_sum=poly_sum+delt_sigma[kk*199:(kk+1)*199]
    return poly_sum/200

In [ ]:
cmp="rbgymc"
font1 = {'family': 'Times New Roman',
         'weight': 'normal',
         'size': 15,
         }
font2 = {'family': 'Times New Roman',
         'weight': 'normal',
         'size': 12,
         }
matplotlib.rcParams['xtick.direction'] = "in"
matplotlib.rcParams['ytick.direction'] = "in"
plt.rcParams['figure.dpi'] = 500
figsize=(6,4)
figure, ax = plt.subplots(figsize=figsize)
plt.xlabel('Strain',font1)
plt.ylabel('Stress',font1)
labels = ax.get_xticklabels() + ax.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]
#plt.xticks(np.linspace(-5, -3, 5))
#plt.yticks(np.linspace(0, 100, 11))
#plt.axis((-0, 0.0003, 0, 300000))
plt.tick_params(labelsize=10)
plt.axis((-0.001, 0.03, 0, 180))
kk=0
for t in [0,4,8,13]:
    text_x=(Variable(torch.from_numpy(large_input_new_nor_1_S[t*199*200:(t+1)*199*200]).to(torch.float32)))
    plt.plot(poly_strain(large_input_all_np_1_S[:,0][t*199*200:(t+1)*199*200])[0:199:4],
         poly_stress(large_output_new_1_S[t*199*200:(t+1)*199*200])[0:-1][0:199:4],"cD-",
             linewidth=0.2,c=cmp[kk])
    plt.plot(poly_strain(large_input_all_np_1_S[:,0][t*199*200:(t+1)*199*200]),
         poly_stress((model(text_x).detach().numpy())[:,0])[0:-1],
             label=str(temperature_txt[t])+"℃ "+strain_rate_txt[0]+"/s",c=cmp[kk])
    kk=kk+1
plt.legend(loc='center', bbox_to_anchor=(0.82,0.82),prop=font2)